# Basics

This notebook will show some of the basic features of [Firedrake](https://www.firedrakeproject.org), which spyro is built upon and also go over the basics of spyro. These are the kind of things you'd need to know to before you begin to model anything. First, we'll show how to use some of the builtin meshing capabilities in Firedrake and then talk a little bit about how Firedrake represents things on meshes using their concept of Functions.

## Meshes

To perform any finite element simulation, you'll first need a mesh. In spyro, we make use a specific kind of mesh composed of simplices, which is just a fancy word for a mesh composed of triangles in two-dimensions and tetrahedrals in three-dimensions. These meshes offer a number of advantages to model complex geometries in a computational efficient manner by being able to largely vary element sizes. 

Firedrake provides a number of utility/helper functions to generate uniform simplical meshes. You can type
```
help(firedrake.utility_meshes)
```

For example, a uniform mesh of a rectangle ∈ [1 x 2] km with ~100 m sized elements would be created like so: 

In [ ]:
import firedrake
nx, ny = 16, 16
mesh = firedrake.UnitRectangleMesh(nx, ny)

## Note on using external meshes 

2D/3D triangular meshes can also be created with external meshers such as [SeismicMesh](https://github.com/krober10nd/SeismicMesh) and the like and used within Firedrake. We'll go over how to use external meshing tools in a later tutorial. Mesh files should be preferably in the gmsh v2.2 or > version in ASCII text file format. A great way to convert mesh formats if via [meshio](https://pypi.org/project/meshio/). 


## Functions

Scalar and vector-valued functions can be defined on meshes using the concept of functions in Firedrake. 

In order to create a `firedrake.function`, we first need to construct what is called a function space `V`. Since we already have a mesh, now we need to decide what element family and polynomial degree to use. In spyro, we exclusively rely on the element family consisting of continuous piecewise-polynomial functions in each simplex, which is abbreviated to CG for "Continuous Galerkin" or "KMV" for "Kong-Mulder-Veldhuizen". 

Next, we have to make an expression for the function and interpolate it to the function space `V`. The function `firedrake.SpatialCoordinate` returns two symbolic objects `x`, `y` for the coordinates of each point of the mesh. These symbols can be used to define expressions. 

In [ ]:
V = FunctionSpace(mesh, 'CG', 1)
x, y = SpatialCoordinate(mesh)
velocity = conditional(x < 0.5, 1.5, 3.0)
vp = Function(V, name="velocity").interpolate(velocity)
File("simple.pvd").write(vp)

Here we've created a function called `vp` that has the value 1.5 on the left side of the domain (where x < 0.5 coordinate) and 3.0 on the right hand side (where x >= 0.5). We use the `firedrake.interpolate` to interpolate the symbolic expression onto the mesh and then we can write it to disk as a pvd file for visualization in ParaView. 

Note all functions have a method `write`, which enables the user to write the function to disk for analysis and visualization in [ParaView](https://www.paraview.org).

## Conclusion

Spyro uses `Firedrake.Functions` to implement solvers for seismic inversion problems. They can also be used to define input fields for simulations, if those fields have a simple analytical expression. In the next tutorial, we'll learn how to simulate a forward wave simulation on a synethic geometry with a point source and adding absorbing boundary conditions. 

To learn more about Firedrake, you can visit their [documentation](https://www.firedrakeproject.org/documentation.html) or check out some of the [demos](https://www.firedrakeproject.org/notebooks.html).
